In [1]:
!pip install -U weaviate-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.1/598.1 kB 10.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: weaviate-client
    Found existing installation: weaviate-client 4.11.0
    Uninstalling weaviate-client-4.11.0:
      Successfully uninstalled weaviate-client-4.11.0

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


> should be 4.16 or newer

In [2]:
!pip show weaviate-client

Name: weaviate-client
Version: 4.18.1
Summary: A python native Weaviate client
Home-page: https://github.com/weaviate/weaviate-python-client
Author: Weaviate
Author-email: hello@weaviate.io,
License: BSD 3-clause
Location: /Users/scottaskinosie/Documents/weaviate/nopenai/weaviate-workshop/.venv/lib/python3.10/site-packages
Requires: authlib, deprecation, grpcio, httpx, protobuf, pydantic, validators
Required-by: distyll-info


## Test 1 - make sure you can connect

> `TODO` - update the host to a url of your instance

In [3]:
import weaviate
import os
from dotenv import load_dotenv
from weaviate.classes.init import Auth

load_dotenv()
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,        # the default REST port
  grpc_port=50051,  # the default GRPC port
  auth_credentials=Auth.api_key(WEAVIATE_KEY)
)

client.is_ready() # Expected True

True

In [4]:
# Display the available modules - ideally you should be able to see 'generative-openai' and 'text2vec-openai'
client.get_meta()

{'grpcMaxMessageSize': 104858000,
 'hostname': 'http://[::]:8080',
 'modules': {'generative-anthropic': {'documentationHref': 'https://docs.anthropic.com/en/api/getting-started',
   'name': 'Generative Search - Anthropic'},
  'generative-anyscale': {'documentationHref': 'https://docs.anyscale.com/endpoints/overview',
   'name': 'Generative Search - Anyscale'},
  'generative-aws': {'documentationHref': 'https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html',
   'name': 'Generative Search - AWS'},
  'generative-cohere': {'documentationHref': 'https://docs.cohere.com/reference/chat',
   'name': 'Generative Search - Cohere'},
  'generative-databricks': {'documentationHref': 'https://docs.databricks.com/en/machine-learning/foundation-models/api-reference.html#completion-task',
   'name': 'Generative Search - Databricks'},
  'generative-friendliai': {'documentationHref': 'https://docs.friendli.ai/openapi/create-chat-completions',
   'name': 'Generative Search - FriendliAI'},
 

In [5]:
client.close()

## Test 2 - test your OpenAI configuration
> `TODO: 1` - update the `host` to a url of your instance

> `TODO: 2` - update the `base_url` (twice) to your OpenAI url where you host the models

In [6]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")
print(f"OpenAI API Key: {OPENAI_API_KEY[:20]}")
print(f"OpenAI URL: {OPENAI_URL}")

OpenAI API Key: sk-proj-iuwKF1Q94jnW
OpenAI URL: https://api.openai.com


In [7]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

True


In [8]:
from weaviate.classes.config import Configure

if client.collections.exists("TestCollection"):
    client.collections.delete("TestCollection")

client.collections.create(
    name="TestCollection",
    
    # Using local ollama embedding model
    vector_config=Configure.Vectors.text2vec_openai(
        model="text-embedding-3-small",
        # base_url=OPENAI_URL
        base_url="https://api.openai.com"
    ),
    
    # Using local ollama generative model
    generative_config=Configure.Generative.openai(
        model="gpt-4-1106-preview",
        base_url="https://api.openai.com"
    ),
)

### Load and vectorize data

In [9]:
import json

# prepare the sample data
with open("./jeopardy_tiny.json") as file:
    data_10 = json.load(file)

print(json.dumps(data_10[0:2], indent=2))

[
  {
    "Category": "SCIENCE",
    "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
    "Answer": "Liver"
  },
  {
    "Category": "ANIMALS",
    "Question": "It's the only living mammal in the order Proboseidea",
    "Answer": "Elephant"
  }
]


In [10]:
test_collection = client.collections.get("TestCollection")
test_collection.data.insert_many(data_10)

BatchObjectReturn(_all_responses=[UUID('19bc8f33-3ea4-4260-9afa-10a9587273ff'), UUID('9030bfac-ffab-4f72-a419-b3b0da96f310'), UUID('477c55d1-7be5-48fc-ba30-a0818dee6064'), UUID('e4118395-ce4c-4e86-b482-270e73985bfb'), UUID('59fff582-2f85-44c9-89ae-043e213476cb'), UUID('91d64a6c-5d25-4c2e-aa11-4a5f822990cb'), UUID('41f2b162-6bf4-43b2-add5-b75a566acc3c'), UUID('b59cca91-e3a6-4eb8-b7d3-1ac27c0b9c44'), UUID('b5ec40fc-560d-482b-82f2-71232dfa2829'), UUID('8db88ed7-0f94-4414-883d-12ce8dbba39f')], elapsed_seconds=3.0840208530426025, errors={}, uuids={0: UUID('19bc8f33-3ea4-4260-9afa-10a9587273ff'), 1: UUID('9030bfac-ffab-4f72-a419-b3b0da96f310'), 2: UUID('477c55d1-7be5-48fc-ba30-a0818dee6064'), 3: UUID('e4118395-ce4c-4e86-b482-270e73985bfb'), 4: UUID('59fff582-2f85-44c9-89ae-043e213476cb'), 5: UUID('91d64a6c-5d25-4c2e-aa11-4a5f822990cb'), 6: UUID('41f2b162-6bf4-43b2-add5-b75a566acc3c'), 7: UUID('b59cca91-e3a6-4eb8-b7d3-1ac27c0b9c44'), 8: UUID('b5ec40fc-560d-482b-82f2-71232dfa2829'), 9: UUID('8

### Check the data

In [11]:
# check the number of objects - should be 10
len(test_collection)

10

In [12]:
# Show data preview - with vectors
test_collection = client.collections.get("TestCollection")
response = test_collection.query.fetch_objects(
    limit=4,
    include_vector=True
)

for item in response.objects:
  print(item.properties)
  print(item.vector, '\n')

{'answer': 'Liver', 'question': 'This organ removes excess glucose from the blood & stores it as glycogen', 'category': 'SCIENCE'}
{'default': [-0.025687167420983315, -0.01295754685997963, 0.010968899354338646, 0.026234189048409462, -0.01862148754298687, 0.01222818624228239, -0.0019159960793331265, 0.030747106298804283, -0.04809221252799034, 0.0026596018578857183, 0.013493170961737633, -0.05830325931310654, 0.0056069595739245415, 0.021390778943896294, 0.029242800548672676, 0.010028707794845104, 0.0037522183265537024, 0.008632666431367397, 0.011014483869075775, 0.0018775336211547256, 0.007743758149445057, -0.03573866933584213, 0.005208090413361788, -0.025778338313102722, -0.0066497172228991985, -0.035601913928985596, -0.005587015766650438, -0.050189126282930374, -0.0007407568045891821, -0.036034971475601196, 0.08606454730033875, -0.046154849231243134, 0.01859869435429573, 0.021014701575040817, 0.023043235763907433, 0.07070238888263702, 0.07644610852003098, 0.02764732390642166, -0.042553

### Query example – it uses the OpenAI key

In [13]:
response = test_collection.query.near_text(
  query="African animals",
  limit=2
)

for item in response.objects:
  print(item.properties)

{'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'}
{'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'}


### RAG example

In [14]:
response = test_collection.generate.near_text(
    query="African animals",
    limit=2,
    single_prompt="Please translate {question} to german"
)

print("=== Source ===")
for item in response.objects:
    print(item.properties)
    print(item.generative.text)

=== Source ===
{'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'}
Mit einem Gewicht von rund einer Tonne ist das Eland die größte Art dieses Tieres in Afrika.
{'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'}
Es ist das einzige lebende Säugetier in der Ordnung der Rüsseltiere.


In [15]:
client.close()